### Passo 1 - Importar Arquivos e Bibliotecas

In [5]:
import pandas as pd
import smtplib
import email.message
import mimetypes 

from pathlib import Path

In [6]:

sales_df = pd.read_excel('Dataset/Vendas.xlsx')
emails_df = pd.read_csv('Dataset/Emails.csv')
stores_df = pd.read_csv('Dataset/Lojas.csv', sep=';', encoding='latin-1')

emails_df = emails_df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], axis=1)

display(sales_df.head())
display(emails_df.head())
display(stores_df.head())

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,1,Sapato Estampa,1,358,358
1,1,2019-01-01,1,Camiseta,2,180,360
2,1,2019-01-01,1,Sapato Xadrez,1,368,368
3,2,2019-01-02,3,Relógio,3,200,600
4,2,2019-01-02,3,Chinelo Liso,1,71,71


,Unnamed: 0.3,Unnamed: 0.2,Loja,Gerente,E-mail
0,0,0,Iguatemi Esplanada,Helena,cezarfrancisco63+helena@gmail.com
1,1,1,Shopping Midway Mall,Alice,cezarfrancisco63+alice@gmail.com
2,2,2,Norte Shopping,Laura,cezarfrancisco63+laura@gmail.com
3,3,3,Shopping Iguatemi Fortaleza,Manuela,cezarfrancisco63+manuela@gmail.com
4,4,4,Shopping União de Osasco,Valentina,cezarfrancisco63+valentina@gmail.com


,ID Loja,Loja
0,1,Iguatemi Esplanada
1,2,Shopping Midway Mall
2,3,Norte Shopping
3,4,Shopping Iguatemi Fortaleza
4,5,Shopping União de Osasco


### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [7]:
sales_df = sales_df.merge(stores_df, on='ID Loja')
display(sales_df)

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja
0,1,2019-01-01,1,Sapato Estampa,1,358,358,Iguatemi Esplanada
1,1,2019-01-01,1,Camiseta,2,180,360,Iguatemi Esplanada
2,1,2019-01-01,1,Sapato Xadrez,1,368,368,Iguatemi Esplanada
3,21,2019-01-02,1,Camisa Gola V Listrado,2,116,232,Iguatemi Esplanada
4,34,2019-01-02,1,Sapato Listrado,1,363,363,Iguatemi Esplanada
...,...,...,...,...,...,...,...,...
100994,69846,2019-12-25,8,Short Estampa,2,96,192,Salvador Shopping
100995,69846,2019-12-25,8,Tênis Estampa,5,256,1280,Salvador Shopping
100996,69850,2019-12-25,8,Calça Estampa,4,177,708,Salvador Shopping
100997,69972,2019-12-26,8,Terno Liso,3,720,2160,Salvador Shopping


In [8]:
last_date = sales_df['Data'].max()
print(last_date)

2019-12-26 00:00:00


### Passo 3 - Salvar a planilha na pasta de backup

In [9]:
dict_stores = {}
for store in stores_df['Loja']:
    dict_stores[store] = sales_df.loc[sales_df['Loja']==store, :]



In [10]:
backup_path = Path('Stores Files Backup')

folders_backup = backup_path.iterdir()
names_list_backup = [folder.name for folder in folders_backup]

for store in stores_df['Loja']:

    # Checks if the folder does not exist and makes it
    if store not in names_list_backup:
        Path(backup_path/store).mkdir()
    
    # Saves the files in the folder
    file_name = f'{last_date.day}_{last_date.month}_{store}.csv'
    dict_stores[store].to_csv(backup_path/store/file_name)


### Passo 4 - Calcular o indicador para 1 loja

In [11]:
#Target setting.

target_billings_day = 1000
target_billings_year = 1650000
target_product_quantity_day = 4
target_product_quantity_year = 120
target_average_sales_day = 500
target_average_sales_year = 500

In [12]:

store_sales = dict_stores[store]
store_sales_date = store_sales.loc[store_sales['Data']==last_date, :]
# store_sales = store_sales.min()
print(store_sales.info)

<bound method DataFrame.info of        Código Venda       Data  ID Loja             Produto  Quantidade  \
12277             5 2019-01-02       25               Calça           1   
12278            52 2019-01-02       25      Gorro Listrado           4   
12279           100 2019-01-02       25  Camisa Gola V Liso           3   
12280           100 2019-01-02       25        Cinto Xadrez           4   
12281           132 2019-01-02       25         Terno Linho           2   
...             ...        ...      ...                 ...         ...   
16234         69908 2019-12-25       25           Meia Liso           2   
16235         69908 2019-12-25       25              Camisa           1   
16236         69913 2019-12-25       25               Gorro           1   
16237         69913 2019-12-25       25         Cinto Linho           1   
16238         69950 2019-12-25       25         Cinto Linho           1   

       Valor Unitário  Valor Final            Loja  
12277         

In [15]:
for store in dict_stores:

    store_sales = dict_stores[store]
    store_sales_date = store_sales.loc[store_sales['Data']==last_date, :]

    # Store Billing
    year_store_billings = store_sales['Valor Final'].sum()
    day_store_billings = store_sales_date['Valor Final'].sum()

    # How many different products were sold.
    product_quantity_year = len(store_sales['Produto'].unique())
    products_quantity_day = len(store_sales_date['Produto'].unique())

    # Average sale value on the year.
    sales_value_year = store_sales.groupby('Código Venda').sum(['Valor Final'])
    average_sales_year = sales_value_year['Valor Final'].mean()

    # Average sale value on the day.
    value_sales_day = store_sales_date.groupby('Código Venda').sum(['Valor Final'])
    average_sales_day = value_sales_day['Valor Final'].mean()  


    if day_store_billings >= target_average_sales_day: color_bill_day = 'green' 
    else: color_bill_day = 'red'

    if year_store_billings >= target_billings_year: color_bill_year = 'green'
    else: color_bill_year = 'red'

    if products_quantity_day >= target_product_quantity_day: color_qtt_day = 'green'
    else: color_qtt_day = 'red'

    if product_quantity_year >= target_product_quantity_year: color_qtt_year = 'green'
    else: color_qtt_year = 'red'

    if average_sales_day >= target_average_sales_day: color_ticket_day = 'green'
    else: color_ticket_day = 'red'

    if average_sales_year >= target_average_sales_year: color_ticket_year = 'green'
    else: color_ticket_year = 'red'
            

    manager_name = emails_df.loc[emails_df['Loja']==store, 'Gerente'].values[0]
    body = f"""
        <p>Dear Manager, {manager_name}</p>

        <p>Yesterday's result <strong>({last_date.day}/{last_date.month})</strong> from the <strong>{store}</strong> store were:</p>

        <table>
            <tr>
            <th>Index</th>
            <th>Day Value</th>
            <th>Day Target</th>
            <th>Day Scenario</th>
            </tr>
            <tr>
            <td>Billings</td>
            <td style="text-align: center">R${day_store_billings:.2f}</td>
            <td style="text-align: center">R${target_billings_day:.2f}</td>
            <td style="text-align: center"><font color="{color_bill_day}">◙</font></td>
            </tr>
            <tr>
            <td>Product Diversity</td>
            <td style="text-align: center">{products_quantity_day}</td>
            <td style="text-align: center">{target_product_quantity_day}</td>
            <td style="text-align: center"><font color="{color_qtt_day}">◙</font></td>
            </tr>
            <tr>
            <td>Average Sales Value</td>
            <td style="text-align: center">R${average_sales_day:.2f}</td>
            <td style="text-align: center">R${target_average_sales_day:.2f}</td>
            <td style="text-align: center"><font color="{color_ticket_day}">◙</font></td>
            </tr>
        </table>
        <br>
        <table>
            <tr>
            <th>Index</th>
            <th>Year Value</th>
            <th>Year Target</th>
            <th>Year Scenario</th>
            </tr>
            <tr>
            <td>Billings</td>
            <td style="text-align: center">R${year_store_billings:.2f}</td>
            <td style="text-align: center">R${target_billings_year:.2f}</td>
            <td style="text-align: center"><font color="{color_bill_year}">◙</font></td>
            </tr>
            <tr>
            <td>Product Diversity</td>
            <td style="text-align: center">{product_quantity_year}</td>
            <td style="text-align: center">{target_product_quantity_year}</td>
            <td style="text-align: center"><font color="{color_qtt_year}">◙</font></td>
            </tr>
            <tr>
            <td>Average Sales Value</td>
            <td style="text-align: center">R${average_sales_year:.2f}</td>
            <td style="text-align: center">R${target_average_sales_year:.2f}</td>
            <td style="text-align: center"><font color="{color_ticket_year}">◙</font></td>
            </tr>
        </table>

        <p>Please find attached the spreadsheet with all the data for more details.</p>

        <p>Yours sincerely</p>
        <p>Francsico</p>
        """

    msg = email.message.EmailMessage()
    msg['Subject'] = f'OnePage of the Day {last_date.day}/{last_date.month} - Loja {store}'
    msg['From'] = '<e-mail>'
    msg['To'] = emails_df.loc[emails_df['Loja']==store, 'E-mail'].values[0]
    msg.add_header('Content-Type', 'text/html')
    msg.set_payload(body)
    password = '<password>' 

    mime_type, _ = mimetypes.guess_type(f'{store}.csv')
    mime_type, mime_subtype = mime_type.split('/')

    file_path  = Path.cwd() / backup_path / store / f'{last_date.day}_{last_date.month}_{store}.csv'

    with open(file_path, 'rb') as file:
        msg.add_attachment(file.read(),
        maintype=mime_type,
        subtype=mime_subtype,
        filename=f'{store}.csv')

    server = smtplib.SMTP('smtp.gmail.com: 587')
    server.starttls()
    # Login Credentials for sending the mail
    server.login(msg['From'], password)
    server.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))


    print(f'E-mail from the {store} store sent!')

E-mail from the Iguatemi Esplanada store sent!
E-mail from the Shopping Midway Mall store sent!
E-mail from the Norte Shopping store sent!
E-mail from the Shopping Iguatemi Fortaleza store sent!
E-mail from the Shopping União de Osasco store sent!
E-mail from the Shopping Center Interlagos store sent!
E-mail from the Rio Mar Recife store sent!
E-mail from the Salvador Shopping store sent!
E-mail from the Rio Mar Shopping Fortaleza store sent!
E-mail from the Shopping Center Leste Aricanduva store sent!
E-mail from the Ribeirão Shopping store sent!
E-mail from the Shopping Morumbi store sent!
E-mail from the Parque Dom Pedro Shopping store sent!
E-mail from the Bourbon Shopping SP store sent!
E-mail from the Palladium Shopping Curitiba store sent!
E-mail from the Passei das Águas Shopping store sent!
E-mail from the Center Shopping Uberlândia store sent!
E-mail from the Shopping Recife store sent!
E-mail from the Shopping Vila Velha store sent!
E-mail from the Shopping SP Market store s

### Passo 5 - Enviar por e-mail para o gerente

In [16]:
# Billing ranking of all stores.

total_billings = sales_df.groupby('Loja')[['Loja', 'Valor Final']].sum('Valor Final')
annual_billings = total_billings.sort_values(by='Valor Final', ascending=False)

annual_rk_file = f'{last_date.day}_{last_date.month}_Annual Ranking.csv'
annual_billings.to_csv(f'Stores Files Backup/{annual_rk_file}')

store_sales_date = sales_df.loc[sales_df['Data']==last_date, :]
stores_billings_day = store_sales_date.groupby('Loja')[['Loja', 'Valor Final']].sum('Valor Final')
stores_billings_day = stores_billings_day.sort_values(by='Valor Final', ascending=False)

days_rk_file = f"{last_date.day}_{last_date.month}_Days Ranking.csv"
stores_billings_day.to_csv(f'Stores Files Backup/{days_rk_file}')



### Passo 6 - Automatizar todas as lojas

### Passo 7 - Criar ranking para diretoria

### Passo 8 - Enviar e-mail para diretoria

In [18]:
# Sending an e-mail to board of directors.
year_best_store = f'R${annual_billings.iloc[0, 0]:,.2f}'.replace(',', '_').replace('.', ',').replace('_', '.')
year_worse_store = f'R${annual_billings.iloc[-1, 0]:,.2f}'.replace(',', '_').replace('.', ',').replace('_', '.')

day_best_store = f'R${stores_billings_day.iloc[0, 0]:,.2f}'.replace(',', '_').replace('.', ',').replace('_', '.')
day_worse_store = f'R${stores_billings_day.iloc[-1, 0]:,.2f}'.replace(',', '_').replace('.', ',').replace('_', '.')

body = f"""
    <p>To whom it may concern,</p>

    <p>Best Store of the Day in Billing: <strong>Loja {stores_billings_day.index[0]}</strong> with revenues of <strong>{day_best_store}</strong></p>
    <p>Worst Store of the Day in Billing: <strong>Loja {stores_billings_day.index[-1]}</strong> with revenues of <strong>{day_worse_store}</strong></p>
    <br>
    <p>Best Store of the Year in Billing: <strong>Loja {annual_billings.index[0]}</strong> with revenues of <strong>{year_best_store}</strong></p>
    <p>Worst Store of the Year in Billing: <strong>Loja {annual_billings.index[-1]}</strong> with revenues of <strong>{year_worse_store}</strong></p>
    <br>
    <p>Please find attached the year and day ratings of all stores.</p>
    <br>
    <p>Yours sincerely</p>
    <p>Francsico</p>
    """

msg = email.message.EmailMessage()
msg['Subject'] = f'Store Ratings {last_date.day}/{last_date.month}'
msg['From'] = '<e-mail>'
msg['To'] = emails_df.loc[emails_df['Loja']=='Diretoria', 'E-mail'].values[0]
msg.add_header('Content-Type', 'text/html')
msg.set_payload(body)
password = '<password>' 


mime_type, _ = mimetypes.guess_type('Days Ranking.csv')
mime_type, mime_subtype = mime_type.split('/')

annual_rk_path  = Path.cwd() / backup_path / f'{last_date.day}_{last_date.month}_Annual Ranking.csv'
days_rk_path  = Path.cwd() / backup_path / f'{last_date.day}_{last_date.month}_Days Ranking.csv'


with open(annual_rk_path, 'rb') as file:
    msg.add_attachment(file.read(),
    maintype=mime_type,
    subtype=mime_subtype,
    filename='Annual Ranking.csv')

with open(days_rk_path, 'rb') as file:
    msg.add_attachment(file.read(),
    maintype=mime_type,
    subtype=mime_subtype,
    filename='Days Ranking.csv')


server = smtplib.SMTP('smtp.gmail.com: 587')
server.starttls()
# Login Credentials for sending the mail
server.login(msg['From'], password)
server.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))


print(f'E-mail to board of directors sent!')

E-mail to board of directors sent!
